In [65]:
import importlib.machinery
import os

# Get the current working directory
cwd = os.getcwd()
print(cwd)
# Define the module name and full path

module_name = "utils"
function_name = "hello"
file_path = os.path.join(cwd, "Lesson_04", "utils", f"{module_name}.py")

# Load the module
try:
    loader = importlib.machinery.SourceFileLoader(module_name, file_path)
    module = loader.load_module()
    hello = getattr(module, function_name)
except:
    file_path = os.path.join(cwd, "utils", f"{module_name}.py")
    loader = importlib.machinery.SourceFileLoader(module_name, file_path)
    module = loader.load_module()
    hello = getattr(module, function_name)

hello("test complete...")
print(file_path)
simulate_bankruptcy = getattr(module, "simulate_bankruptcy")
create_dashboard = getattr(module, "create_dashboard")


/home/cooneycw/jupyter-workspace
test complete...
/home/cooneycw/jupyter-workspace/Lesson_04/utils/utils.py


In [66]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import VBox, HBox, Label, Output
from ipywidgets import interact, FloatSlider, IntSlider
import pandas as pd


# Run a single simulation with default parameters
bankruptcy_times = simulate_bankruptcy()
print(f"Average time to bankruptcy: {np.mean(bankruptcy_times):.2f} years")
print(f"Median time to bankruptcy: {np.median(bankruptcy_times):.2f} years")


Average time to bankruptcy: 5.20 years
Median time to bankruptcy: 5.00 years


In [67]:
# Cell 4: Create the interactive dashboard
create_dashboard()

interactive(children=(FloatSlider(value=5.0, continuous_update=False, description='Claims/Year:', max=20.0, mi…

In [68]:
# Cell 5: Create a custom interactive widget for parameter exploration
from ipywidgets import interact_manual, FloatSlider, IntSlider

@interact_manual(
    initial_capital=IntSlider(min=100000, max=5000000, step=100000, value=1000000, description='Initial $:'),
    claim_rate=FloatSlider(min=1, max=20, step=0.5, value=5, description='Claims/Year:'),
    avg_claim_size=FloatSlider(min=10000, max=200000, step=5000, value=50000, description='Avg Claim $:'),
    investment_return=FloatSlider(min=0.0, max=0.15, step=0.01, value=0.05, description='Return %:'),
    num_simulations=IntSlider(min=10, max=1000, step=10, value=100, description='Simulations:')
)
def run_custom_simulation(initial_capital, claim_rate, avg_claim_size, investment_return, num_simulations):
    # Run the simulation with custom parameters
    bankruptcy_times = simulate_bankruptcy(
        initial_capital=initial_capital,
        claim_rate=claim_rate,
        avg_claim_size=avg_claim_size,
        investment_return=investment_return,
        num_simulations=num_simulations
    )

    # Create visualization
    fig, ax = plt.subplots(1, 1, figsize=(10, 6))

    # Create histogram with cumulative distribution
    n, bins, patches = ax.hist(bankruptcy_times, bins=30, alpha=0.7, edgecolor='black', label='Frequency')
    ax2 = ax.twinx()
    ax2.hist(bankruptcy_times, bins=30, cumulative=True, histtype='step',
             color='red', linewidth=2, label='Cumulative', density=True)

    ax.set_xlabel('Years until Bankruptcy')
    ax.set_ylabel('Frequency')
    ax2.set_ylabel('Cumulative Probability')
    ax.set_title(f'Bankruptcy Simulation Results\n(Mean: {np.mean(bankruptcy_times):.1f} years, Median: {np.median(bankruptcy_times):.1f} years)')

    # Add statistics text box
    stats_text = f"""Statistics:
Mean: {np.mean(bankruptcy_times):.1f} years
Median: {np.median(bankruptcy_times):.1f} years
Std Dev: {np.std(bankruptcy_times):.1f} years
Min: {np.min(bankruptcy_times):.1f} years
Max: {np.max(bankruptcy_times):.1f} years
Prob(survive 10y): {(bankruptcy_times > 10).mean():.2%}"""

    ax.text(0.72, 0.98, stats_text, transform=ax.transAxes,
            bbox=dict(boxstyle="round,pad=0.5", facecolor="wheat", alpha=0.8),
            verticalalignment='top', fontsize=10)

    ax.legend(loc='upper left')
    ax2.legend(loc='upper center')
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()


interactive(children=(IntSlider(value=1000000, description='Initial $:', max=5000000, min=100000, step=100000)…

In [69]:
# Cell 6: Create a parameter sensitivity analysis with dynamic sliders
def sensitivity_analysis():
    # Create output widget
    output = Output()

    # Define parameter ranges dictionary
    parameter_ranges = {
        'claim_rate': {
            'min': 1.0,
            'max': 20.0,
            'step': 0.5,
            'default_min': 2.0,
            'default_max': 10.0,
            'description': 'Claims/Year',
            'unit': 'claims/year'
        },
        'avg_claim_size': {
            'min': 10000.0,
            'max': 200000.0,
            'step': 5000.0,
            'default_min': 20000.0,
            'default_max': 100000.0,
            'description': 'Average Claim Size',
            'unit': '$'
        },
        'investment_return': {
            'min': 0.0,
            'max': 0.15,
            'step': 0.01,
            'default_min': 0.02,
            'default_max': 0.10,
            'description': 'Investment Return',
            'unit': '%'
        }
    }

    # Create parameter selection dropdown
    parameter_name = widgets.Dropdown(
        options=['claim_rate', 'avg_claim_size', 'investment_return'],
        value='claim_rate',
        description='Parameter:'
    )

    # We'll create a container for the sliders that we can replace entirely
    slider_container = VBox()

    # Create run button
    run_button = widgets.Button(description='Run Analysis')

    # Create number of points slider (this doesn't need to change)
    num_points = IntSlider(min=3, max=20, step=1, value=8, description='# Points:')

    # Function to create new sliders based on the selected parameter
    def create_sliders_for_parameter(param_name):
        param_info = parameter_ranges[param_name]

        # Create fresh sliders with the correct ranges
        min_slider = FloatSlider(
            min=param_info['min'],
            max=param_info['max'],
            step=param_info['step'],
            value=param_info['default_min'],
            description=f'Min {param_info["description"]}:',
            continuous_update=False
        )

        max_slider = FloatSlider(
            min=param_info['min'],
            max=param_info['max'],
            step=param_info['step'],
            value=param_info['default_max'],
            description=f'Max {param_info["description"]}:',
            continuous_update=False
        )

        return min_slider, max_slider

    # Create initial sliders
    min_value, max_value = create_sliders_for_parameter(parameter_name.value)

    # Update the slider container
    slider_container.children = [
        HBox([min_value]),
        HBox([max_value])
    ]

    # Function to handle parameter changes
    def on_parameter_change(change):
        if change['name'] == 'value':
            # Create new sliders for the selected parameter
            new_min, new_max = create_sliders_for_parameter(change['new'])

            # Replace the old sliders with new ones
            slider_container.children = [
                HBox([new_min]),
                HBox([new_max])
            ]

            # Update our references
            nonlocal min_value, max_value
            min_value = new_min
            max_value = new_max

    # Set up observer for parameter dropdown
    parameter_name.observe(on_parameter_change, names='value')

    def run_sensitivity_analysis(button):
        parameter = parameter_name.value
        min_val = min_value.value
        max_val = max_value.value
        n_points = num_points.value

        with output:
            output.clear_output()

            # Validation
            if min_val >= max_val:
                print("Error: Minimum value must be less than maximum value!")
                return

            print(f"Running sensitivity analysis for: {parameter}")
            print(f"Range: {min_val:,.2f} to {max_val:,.2f}")
            print(f"Testing {n_points} values...")
            print("-" * 50)

            # Generate parameter values
            param_values = np.linspace(min_val, max_val, n_points)
            results = []

            # Base parameters
            base_params = {
                'claim_rate': 5,
                'avg_claim_size': 50000,
                'investment_return': 0.05,
                'num_simulations': 100
            }

            # Run simulations for each parameter value
            for i, value in enumerate(param_values):
                print(f"Progress: {i+1}/{n_points} simulations... ({value:,.2f})", end='\r')
                params = base_params.copy()
                params[parameter] = value

                bankruptcy_times = simulate_bankruptcy(**params)
                results.append({
                    'value': value,
                    'mean_bankruptcy_time': np.mean(bankruptcy_times),
                    'median_bankruptcy_time': np.median(bankruptcy_times),
                    'std_bankruptcy_time': np.std(bankruptcy_times)
                })

            print(f"Progress: Complete! All {n_points} simulations finished.    ")
            print("-" * 50)

            df_results = pd.DataFrame(results)

            # Plot results
            fig, ax = plt.subplots(1, 1, figsize=(10, 6))

            # Adjust plotting based on parameter type
            if parameter == 'investment_return':
                x_values = df_results['value'] * 100  # Convert to percentage
                ax.errorbar(x_values, df_results['mean_bankruptcy_time'],
                           yerr=df_results['std_bankruptcy_time'], fmt='o-', capsize=5,
                           label='Mean ± Std Dev', markersize=8)
                ax.plot(x_values, df_results['median_bankruptcy_time'],
                       's--', label='Median', markersize=8)
                ax.set_xlabel(f'{parameter_ranges[parameter]["description"]} (%)')
            else:
                ax.errorbar(df_results['value'], df_results['mean_bankruptcy_time'],
                           yerr=df_results['std_bankruptcy_time'], fmt='o-', capsize=5,
                           label='Mean ± Std Dev', markersize=8)
                ax.plot(df_results['value'], df_results['median_bankruptcy_time'],
                       's--', label='Median', markersize=8)

                if parameter == 'avg_claim_size':
                    ax.set_xlabel(f'{parameter_ranges[parameter]["description"]} ($)')
                    ax.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x:,.0f}'))
                else:
                    ax.set_xlabel(f'{parameter_ranges[parameter]["description"]}')

            ax.set_ylabel('Years until Bankruptcy')
            ax.set_title(f'Sensitivity Analysis: {parameter_ranges[parameter]["description"]}')
            ax.legend()
            ax.grid(True, alpha=0.3)

            plt.tight_layout()
            plt.show()

            # Create a formatted data table
            df_display = df_results.copy()

            if parameter == 'investment_return':
                df_display['value'] = df_display['value'] * 100  # Convert to percentage
                df_display = df_display.round(2)
                df_display = df_display.rename(columns={'value': 'Return (%)'})
            elif parameter == 'avg_claim_size':
                df_display['value'] = df_display['value'].apply(lambda x: f'${x:,.0f}')
                df_display = df_display.rename(columns={'value': 'Claim Size'})
            else:
                df_display = df_display.round(2)
                df_display = df_display.rename(columns={'value': 'Claims/Year'})

            # Round other columns for display
            for col in ['mean_bankruptcy_time', 'median_bankruptcy_time', 'std_bankruptcy_time']:
                df_display[col] = df_display[col].round(2)

            print("\nSensitivity Analysis Results:")
            display(df_display)

    # Connect button to function
    run_button.on_click(run_sensitivity_analysis)

    # Create layout
    controls = VBox([
        HBox([parameter_name]),
        slider_container,  # This contains our dynamic sliders
        HBox([num_points]),
        run_button
    ])

    # Display interface
    display(controls)
    display(output)

In [70]:
# Run the sensitivity analysis
sensitivity_analysis()

Output()